<a href="https://colab.research.google.com/github/pcpiscator/2T2021/blob/main/C%C3%B3pia_de_Furg_ECD_Machine_Learning_II_Semana_06_Sistemas_de_recomenda%C3%A7%C3%A3o_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning II - Sistemas de recomendação I
### Prof. Marcelo Malheiros

Parte do código adaptada de Aditya Sharma (Datacamp)

---

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_:

- NumPy: suporte a vetores, matrizes e operações de Álgebra Linear
- Matplotlib: biblioteca de visualização de dados
- Pandas: pacote estatístico e de manipulação de DataFrames
- Scikit-Learn: biblioteca com algoritmos de Machine Learning

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

# Conjunto de dados

**Atenção:** é preciso fazer o _upload_ do arquivo **popular_movies_csv.zip** primeiro.

Para este _notebook_ vamos usar uma versão condensada do _dataset_ chamado [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset/).

A versão completa contém informações detalhadas sobre 45.000 filmes, além de 29 milhões de avaliações destes filmes, feitas por 270 mil usuários. As avaliações seguem a métrica usual de estrelas, em uma escala de 1 a 5.

A versão condensada contém a maioria dos metadados sobre os 4.606 filmes mais populares, incluindo atributos como título, ano, duração, nome do diretor e popularidade. As avaliações individuais de usuários não fazem parte deste _dataset_, uma vez que iremos focar em **recomendações baseadas em conteúdo**.


In [3]:
# se o arquivo ZIP contiver um único CSV, este pode ser carregado diretamente
#filmes = pd.read_csv('popular_movies_csv.zip', index_col=0)

In [4]:
filmes = pd.read_csv('https://raw.githubusercontent.com/pcpiscator/2T2021/main/popular_movies.csv')

In [6]:
filmes.head(3)

,Unnamed: 0,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,cast,keywords,director
0,0,25000000,"['Drama', 'Crime']",NaN,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,51.645403,1994-09-23,28341469.0,142.0,Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.5,8358.0,"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']","['prison', 'corruption', 'police brutality']",Frank Darabont
1,1,6000000,"['Drama', 'Crime']",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",41.109264,1972-03-14,245066411.0,175.0,Released,An offer you can't refuse.,The Godfather,False,8.5,6024.0,"['Marlon Brando', 'Al Pacino', 'James Caan']","['italy', 'love at first sight', 'loss of fath...",Francis Ford Coppola
2,2,13200000,"['Comedy', 'Drama', 'Romance']",NaN,19404,tt0112870,hi,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",34.457024,1995-10-20,100000000.0,190.0,Released,Come... Fall In Love,Dilwale Dulhania Le Jayenge,False,9.1,661.0,"['Shah Rukh Khan', 'Kajol', 'Amrish Puri']",['musical'],Aditya Chopra


In [7]:
filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4606 entries, 0 to 4605
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         4606 non-null   int64  
 1   budget             4606 non-null   int64  
 2   genres             4606 non-null   object 
 3   homepage           1847 non-null   object 
 4   id                 4606 non-null   int64  
 5   imdb_id            4606 non-null   object 
 6   original_language  4606 non-null   object 
 7   original_title     4606 non-null   object 
 8   overview           4601 non-null   object 
 9   popularity         4606 non-null   float64
 10  release_date       4606 non-null   object 
 11  revenue            4606 non-null   float64
 12  runtime            4606 non-null   float64
 13  status             4606 non-null   object 
 14  tagline            4102 non-null   object 
 15  title              4606 non-null   object 
 16  video              4606 

# Recomendações baseadas em conteúdo

Este tipo de sistema de recomendação foca em analisar os **metadados** dos itens para gerar indicações, porém sem utilizar neste processo o cruzamento entre perfis de usuários.

Podemos separar esta abordagem para sistemas de recomendação em dois tipos:

- **Recomendações não-personalizadas**, em que apenas métricas mais gerais são usadas para sugerir novos itens.

- **Recomendações personalizadas**, em que o perfil do usuário é usado para medir similaridade entre itens e assim sugerir novos itens.

# Recomendações não-personalizadas

## Recomendação por popularidade

Este é o tipo mais simples de recomendação que pode ser construída, pois usa apenas medidas globais calculadas sobre os itens disponíveis.

Aqui, por exemplo, vamos elencar mais bem avaliados (globalmente, sem levar em conta a avaliação individual de usuários) e os filmes com mais votos (sem levar em conta a avaliação).

In [8]:
# média de avaliações
m_aval = filmes['vote_average'].mean()
print(m_aval)

6.474924012158065


In [9]:
# top 10 dos filmes melhor avaliados
filmes.sort_values(by=['vote_average'], ascending=False).head(10).title

2          Dilwale Dulhania Le Jayenge
314                       Planet Earth
0             The Shawshank Redemption
16                          Your Name.
1                        The Godfather
10              The Godfather: Part II
37         Once Upon a Time in America
450      Black Mirror: White Christmas
11     One Flew Over the Cuckoo's Nest
14                              Psycho
Name: title, dtype: object

In [10]:
# média do número de votos por filme
m_votos = filmes['vote_count'].mean()
print(m_votos)

919.726443768997


In [11]:
# top 10 dos filmes mais votados
filmes.sort_values(by=['vote_count'], ascending=False).head(10).title

21                   Inception
3              The Dark Knight
413                     Avatar
251               The Avengers
252                   Deadpool
22                Interstellar
82            Django Unchained
51     Guardians of the Galaxy
4                   Fight Club
807           The Hunger Games
Name: title, dtype: object

## Recomendação por avaliação ponderada

Usar contagens simples de avaliação ou números de votos pode distorcer a representatividade de um certo item. Por exemplo, um filme votado por poucos mas bem avaliado pode ter mais destaque do que um filme bem popular mas que teve uma avaliação ligeiramente menor.

O ideal é criar uma medida ponderada, que chamaremos de **score**, que combina tanto o número de votos recebidos como a avaliação dada. Isso é uma indicação melhor da qualidade percebida: quanto maior o número de votos, mais peso terá a avaliação recebida.

In [13]:
# medida de avaliação ponderada, segundo critério do IMDB
def avaliação_ponderada(item):
    aval = item['vote_average']
    votos = item['vote_count']
    return (votos / (votos + m_votos) * aval) + (m_votos / (m_votos + votos) * m_aval)

In [14]:
# define novo atributo 'score'
filmes['score'] = filmes.apply(avaliação_ponderada, axis=1)

In [15]:
# novo DataFrame com base no 'score'
top = filmes.sort_values('score', ascending=False)

In [16]:
# top 10 dos filmes por 'score'
top[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
0,The Shawshank Redemption,8358.0,8.5,8.299249
1,The Godfather,6024.0,8.5,8.231770
3,The Dark Knight,12269.0,8.3,8.172727
4,Fight Club,9678.0,8.3,8.141610
5,Pulp Fiction,8670.0,8.3,8.124962
12,Forrest Gump,8147.0,8.2,8.025009
21,Inception,14075.0,8.1,8.000323
6,Schindler's List,4436.0,8.3,7.986584
7,Whiplash,4376.0,8.3,7.983033
22,Interstellar,11187.0,8.1,7.976546


# Recomendações personalizadas

Recomendações personalizadas são calculadas com base nos **metadados**, como antes, mas levam em conta muito mais atributos.

Além disso, usam **conhecimento prévio sobre o usuário** para selecionar itens potencialmente mais relevantes para o mesmo.

## Extração de novas _features_

Em um primeiro momento vamos indexar mais metadados, em especial usar agora **palavras-chave** extraídas de diversos campos de texto.

In [17]:
# resumo do enredo dos primeiros filmes
filmes['overview'].head(5)

0    Framed in the 1940s for the double murder of h...
1    Spanning the years 1945 to 1955, a chronicle o...
2    Raj is a rich, carefree, happy-go-lucky second...
3    Batman raises the stakes in his war on crime. ...
4    A ticking-time-bomb insomniac and a slippery s...
Name: overview, dtype: object

Para extrair palavras-chave, precisamos usar várias técnicas de Processamento de Linguagem Natural (PLN).

Para isso iremos usar o algoritmo vetorizador `TfidfVectorizer` da biblioteca Sciki-Learn, que implementa a técnica Term Frequency – Inverse Document Frequency (TFIDF). 

Um vetorizador é um processo que mede a importância de cada palavra para um documento, em relação a todas as demais palavras de uma coleção de documentos.

Cada medida de importância é usada como um fator de ponderação em pesquisas de recuperação de informações e mineração de textos. O valor **TF-IDF** aumenta proporcionalmente ao número de vezes em que uma palavra aparece no documento, sendo compensado pelo número de documentos na coleção que contém tal palavra, ajustando o fato de que algumas palavras aparecem com mais frequência em geral.

In [19]:
# importação
from sklearn.feature_extraction.text import TfidfVectorizer

Um ponto fundamental é que estamos **processando a linguagem humana**, então é preciso conhecimento prévio sobre a língua dos textos que estamos manipulando.

Em especial, precisamos ter uma lista de palavras especiais chamadas **stopwords**, que são palavras frequentes daquela língua e que podem ser removidas de um texto sem prejuízo para a identificação de elementos importantes.

Tipicamente a lista de _stopwords_ inclui artigos, pronomes, conjunções e outros elementos estruturais das frases.

In [20]:
# criação do modelo TF-IDF para a língua inglesa
tfidf = TfidfVectorizer(stop_words='english')

In [21]:
# troca valores ausentes em alguns resumos pela string vazia
filmes['overview'] = filmes['overview'].fillna('')

**Atenção:** O algoritmo TF-IDF cria muitos novos atributos para cada instância, então é preciso cautela com o tamanho do conjunto de dados a ser processado para efeitos de consumo de memória.

Basicamente, para cada termo indexado haverá uma nova coluna, indicando a presença ou não do mesmo.

In [22]:
# dados originais
filmes.shape

(4606, 23)

In [23]:
# criação da matriz TF-IDF com base no atributo 'overview'
matriz_tfidf = tfidf.fit_transform(filmes['overview'])

In [24]:
# novos termos indexados
len(tfidf.get_feature_names())

19694

In [27]:
# lista de alguns termos indexados
tfidf.get_feature_names()[4000:4010]

['copilot',
 'coping',
 'copper',
 'copperbottom',
 'cops',
 'copy',
 'copying',
 'copywriter',
 'cora',
 'coraline']

In [28]:
# matriz TF-IDF
matriz_tfidf.shape

(4606, 19694)

## Critério de similaridade

Sistemas de recomendação precisam definir um critério de **similaridade** entre itens para encontrar e sugerir os mais relevantes.

Aqui vamos definir a similaridade entre dois filmes usando o **coeficiente do cosseno**, também chamada de similaridade por cosseno (_cosine similarity_). A ideia é medir o ângulo em um espaço de muitas dimensões, de forma que um ângulo menor indica mais similaridade e um ângulo maior, menos similaridade.

Como estamos usando o vetorizador TF-IDF, que produz medidas normalizadas, é mais eficiente usar `linear_kernel()` para calcular cada medida entre pares de instâncias, ao invés da função mais genérica `cosine_similarities()` (que também está disponível mas seria mais lenta neste caso).

In [29]:
# importação
from sklearn.metrics.pairwise import linear_kernel

In [30]:
# calcular a matriz de similaridade por cosseno
matriz_cosseno = linear_kernel(matriz_tfidf, matriz_tfidf)

In [31]:
matriz_cosseno.shape

(4606, 4606)

In [32]:
matriz_cosseno[0]

array([1.        , 0.00523362, 0.00949937, ..., 0.        , 0.        ,
       0.        ])

## Recomendação baseada em similaridade

A ideia é, dado o nome de um filme, obter a indicação de títulos de filmes com grande similaridade (usando a `matriz_cosseno` criada anteriormente).

Para isso, precisamos primeiro de um mapeamento reverso de títulos de filmes para índices do DataFrame `filmes`.

In [33]:
# mapeamento reverso
índice = pd.Series(filmes.index, index=filmes['title'])

In [34]:
índice.shape

(4606,)

In [35]:
índice.head(10)

title
The Shawshank Redemption       0
The Godfather                  1
Dilwale Dulhania Le Jayenge    2
The Dark Knight                3
Fight Club                     4
Pulp Fiction                   5
Schindler's List               6
Whiplash                       7
Spirited Away                  8
Life Is Beautiful              9
dtype: int64

Agora podemos definir a função de recomendação.

Note que para ela funcionar, é preciso indicar o **título exato** de um filme que já esteja entre os itens disponíveis.

Para um sistema mais complexo poderia ser necessária uma outra função

In [36]:
# função que recebe o título do filme e retorna os 10 filmes mais similares
def recomendações(título, similaridade):
    # obtenha o índice de um filme dado seu título (usando o mapeamento reverso)
    if título not in índice:
        raise Exception('Filme não encontrado')
    idx = índice[título]

    # obter a lista de pontuações de similaridade associadas a este filme (em relação a todos os outros)
    sim_scores = list(enumerate(similaridade[idx]))

    # ordenar a lista scores por similaridade decrescente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # obter os 10 mais similares (ignorando o primeiro, pois é o próprio filme consultado)
    movie_indices = [i[0] for i in sim_scores[1:11]]

    # retornar os títulos dos 10 mais similares
    return filmes['title'].loc[movie_indices]

In [37]:
recomendações('The Godfather', matriz_cosseno)

10       The Godfather: Part II
3444                 Blood Ties
672     The Godfather: Part III
2865              Live by Night
1151                   Sinister
4486      The Cold Light of Day
2340                        Joe
505           Road to Perdition
1384         Death at a Funeral
1543          The Addams Family
Name: title, dtype: object

In [38]:
recomendações('The Dark Knight', matriz_cosseno)

146                       The Dark Knight Rises
1588                             Batman Returns
4403                             Batman Forever
503                  Batman: Under the Red Hood
324     Batman: The Dark Knight Returns, Part 2
469     Batman: The Dark Knight Returns, Part 1
768                                      Batman
1553                           Batman: Year One
2817                   Batman: The Killing Joke
1259               Batman: Mask of the Phantasm
Name: title, dtype: object

## Busca aproximada por título

Muitas vezes a entrada de dados vem do usuário ou de uma outra base, o que pode ter inconsistências na escrita. Isso vale também para os títulos dos filmes.

Então aqui podemos fazer uma **busca aproximada** por título, de forma a encontrar um título válido da base mesmo que a entrada não tenha sido exata.

O processo na prática é o mesmo de definir medidas de similaridade entre itens, e então selecionar o mais semelhante. Como isso se aplica a _strings_, felizmente já podemos utilizar funções prontas de bibliotecas que fazem todo o processo de forma bem direta e eficiente.

O trecho de código abaixo usa uma biblioteca chamada `fuzzywuzzy`. Como ela não vem instalada por padrão no Colaboratory, é preciso primeiro solicitar a instalação, tal como na linha abaixo:

In [39]:
! pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 50 kB 2.8 MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149867 sha256=39b26ca8369e5fbc662f31ea22612d68197401faf3827962f971fd03ede3498b
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein


O comando acima também funciona no ambiente Anaconda. Mas uma forma alternativa de instalação usa a seguinte linha de comando:

    conda install -c conda-forge fuzzywuzzy

In [40]:
# importação
from fuzzywuzzy import fuzz, process

In [41]:
# exemplo de medida de similaridade
fuzz.ratio('Isso é um teste!', 'Isso é  um teste')

94

In [42]:
# lista de títulos para a busca aproximada
títulos = filmes.title

In [43]:
# indicação de títulos encontrados
process.extract('alien', títulos, limit=10)

[('Alien', 100, 65),
 ('Alien³', 100, 2565),
 ('Aliens', 91, 130),
 ('Ali', 90, 1827),
 ('How to Lose Friends & Alienate People', 90, 3049),
 ('Monsters vs Aliens', 90, 3073),
 ('Alien: Resurrection', 90, 3309),
 ('Alien: Covenant', 90, 3720),
 ('AVP: Alien vs. Predator', 90, 3956),
 ('Aliens in the Attic', 90, 4214)]

## Refinamento do sistema de recomendação

O critério de similaridade definido anteriormente levava em conta apenas as palavras mais significativas usadas no resumo de cada filme, mas não usava outros metadados.

Assim, alguns filmes em série como a saga "O Poderoso Chefão" (The Godfather) foram recomendados, ao passo que nenhum outro filme do Batman apareceu entre as recomendações.

O resumo do enredo pode ajudar, mas existem outros metadados disponíveis muito mais importantes, e que iremos usar agora. Em especial o próprio título do filme, o diretor, os três atores principais, uma lista de gêneros relacionados e ainda as palavras-chave do enredo do filme (todos esses dados já estão prontos para o uso como atributos de cada instância).

In [44]:
# título
filmes.title[0]

'The Shawshank Redemption'

In [45]:
# diretor
filmes.director[0]

'Frank Darabont'

É necessária apenas a conversão de formato textual em listas para três atributos: elenco (_cast_), gêneros (_genre_) e palavras-chave (_keywords_).

In [46]:
# três atores principais
filmes.cast[0]

"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']"

In [47]:
# gêneros do filme
filmes.genres[0]

"['Drama', 'Crime']"

In [48]:
# palavras-chave
filmes.keywords[0]

"['prison', 'corruption', 'police brutality']"

In [49]:
# função para converter representação em texto para lista
def string_para_lista(x):
    if isinstance(x, str) and x[0] == '[' and x[-1] == ']':
        return eval(x)
    else:
        return x

In [50]:
# aplica conversão para lista em três atributos
features = ['cast', 'genres', 'keywords']
for feature in features:
    filmes[feature] = filmes[feature].apply(string_para_lista)

In [51]:
filmes.cast[0]

['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']

In [52]:
filmes.genres[0]

['Drama', 'Crime']

In [53]:
filmes.keywords[0]

['prison', 'corruption', 'police brutality']

In [54]:
# estes serão os metadados a serem usados agora
filmes[['title', 'director', 'cast', 'genres', 'keywords']].head(5)

,title,director,cast,genres,keywords
0,The Shawshank Redemption,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton]","[Drama, Crime]","[prison, corruption, police brutality]"
1,The Godfather,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan]","[Drama, Crime]","[italy, love at first sight, loss of father]"
2,Dilwale Dulhania Le Jayenge,Aditya Chopra,"[Shah Rukh Khan, Kajol, Amrish Puri]","[Comedy, Drama, Romance]",[musical]
3,The Dark Knight,Christopher Nolan,"[Christian Bale, Michael Caine, Heath Ledger]","[Drama, Action, Crime]","[dc comics, crime fighter, secret identity]"
4,Fight Club,David Fincher,"[Edward Norton, Brad Pitt, Meat Loaf]",[Drama],"[support group, dual identity, nihilism]"


Como estes são campos textuais, podemos simplificar eles, convertendo para minúsculas e removendo os espaços.

In [55]:
# conversão de textos para minúsculas e remoção de espaços
def limpeza(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ', '')) for i in x]
    elif isinstance(x, str):
        return str.lower(x.replace(' ', ''))
    else:
        return ''

In [56]:
# limpeza dos atributos indicados
features = ['director', 'cast', 'genres', 'keywords']
for feature in features:
    filmes[feature] = filmes[feature].apply(limpeza)

In [57]:
# função para criar 'soup' de palavras
def criar_soup(x):
    return x['title'].lower() + ' ' + x['director'] + ' ' + ' '.join(x['cast']) + ' ' + \
           ' '.join(x['genres']) + ' ' + ' '.join(x['keywords'])

In [58]:
# novo atributo
filmes['soup'] = filmes.apply(criar_soup, axis=1)

In [59]:
filmes['soup'][0]

'the shawshank redemption frankdarabont timrobbins morganfreeman bobgunton drama crime prison corruption policebrutality'

Aqui vamos usar o algoritmo `CountVectorizer`, que é um tipo mais simples de vetorizador. Aqui todas as palavras serão contadas e mantidas em uma nova matriz.

In [60]:
# importação
from sklearn.feature_extraction.text import CountVectorizer

# criação do vetorizador
contagem = CountVectorizer()

# matriz de contagem baseada no atributo 'soup'
matriz_contagem = contagem.fit_transform(filmes['soup'])

In [61]:
matriz_contagem.shape

(4606, 13945)

In [62]:
# importação
from sklearn.metrics.pairwise import cosine_similarity

# matriz de similaridade
matriz_cosseno2 = cosine_similarity(matriz_contagem, matriz_contagem)

In [63]:
matriz_cosseno2.shape

(4606, 4606)

Agora basta chamar a mesma função de antes, passando a nova matriz de similaridade

In [64]:
recomendações('The Godfather', matriz_cosseno2)

672        The Godfather: Part III
10          The Godfather: Part II
717               The Conversation
1749                 The Outsiders
2226                 The Rainmaker
321              On the Waterfront
855        The Talented Mr. Ripley
1138    The Place Beyond the Pines
27        The Silence of the Lambs
2552         The Girl on the Train
Name: title, dtype: object

In [65]:
recomendações('The Dark Knight', matriz_cosseno2)

146                       The Dark Knight Rises
199                               Batman Begins
47                                 The Prestige
324     Batman: The Dark Knight Returns, Part 2
469     Batman: The Dark Knight Returns, Part 1
3684                                 The Double
3340                                     Hitman
4050                                 The Gunman
2320                              The Protector
2935                               The Punisher
Name: title, dtype: object

In [66]:
recomendações('Alien', matriz_cosseno2)

130                  Aliens
3309    Alien: Resurrection
3720        Alien: Covenant
2565                 Alien³
2975             Dark Skies
2257           Dog Soldiers
2197             Prometheus
2534                Riddick
3404              Moonraker
4112                   Bait
Name: title, dtype: object